In [1]:
import pandas as pd
import json
import os
import nest_asyncio
from typing import List, Optional, Literal
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup
from crawl4ai import AsyncWebCrawler, CrawlerRunConfig, CacheMode, BrowserConfig
from crawl4ai.async_dispatcher import MemoryAdaptiveDispatcher
from crawl4ai import RateLimiter as CrawlRateLimiter
from pydantic import BaseModel, Field
from google import genai
from google.genai import types
from dotenv import load_dotenv
import re
import asyncio
from aiolimiter import AsyncLimiter

### Filter List

In [70]:
df = pd.read_json('venues.json')

In [71]:
df.head()

,id,name,location,contact,type,tagline,price_range_id,rating,num_ratings,url_slug,enable_discovery,enable_for_amex,deep_link,metadata,web_link
0,"{'resy': 84342, 'foursquare': None, 'google': ...",Like Music VIP Cancún,{'address_1': 'Av Xcaret Supermanzana 35 Manza...,"{'phone_number': None, 'url': None}",Cocktail Bar,None,2,NaN,0.0,like-music-vip-cancun,1,1,resy://resy.com/VenueDetails?venue_id=%7B%27re...,{'description': ' Like Music VIP Cancún is a C...,"https://resy.com/?venue_id={'resy': 84342, 'fo..."
1,"{'resy': 81110, 'foursquare': None, 'google': ...",Restaurante Da Enzo Playa del Carmen,"{'address_1': 'Calle 42 Entre Av. 10 y, 5 Av. ...","{'phone_number': None, 'url': None}",Italian,None,2,NaN,0.0,restaurante-da-enzo-playa-del-carmen,1,1,resy://resy.com/VenueDetails?venue_id=%7B%27re...,{'description': ' Restaurante Da Enzo Playa de...,"https://resy.com/?venue_id={'resy': 81110, 'fo..."
2,"{'resy': 78530, 'foursquare': None, 'google': ...",Restaurante El Plebe Bichi Teotihuacan,"{'address_1': 'Calle Emilio Carranza 222, 5585...","{'phone_number': None, 'url': None}",Seafood,None,2,NaN,0.0,restaurante-el-plebe-bichi-teotihuacan,1,1,resy://resy.com/VenueDetails?venue_id=%7B%27re...,{'description': ' Restaurante El Plebe Bichi T...,"https://resy.com/?venue_id={'resy': 78530, 'fo..."
3,"{'resy': 78730, 'foursquare': None, 'google': ...",Restaurante La Mentirosa Los Mochis,"{'address_1': 'Blvd Centenario 805, Centro, 81...","{'phone_number': None, 'url': None}",International,None,2,NaN,0.0,restaurante-la-mentirosa-los-mochis,1,1,resy://resy.com/VenueDetails?venue_id=%7B%27re...,{'description': ' Restaurante La Mentirosa Los...,"https://resy.com/?venue_id={'resy': 78730, 'fo..."
4,"{'resy': 75788, 'foursquare': None, 'google': ...",Restaurante Salmone's Morelia Suc. Siervo,"{'address_1': 'Av Siervo de La Nacion s/n, Agu...","{'phone_number': None, 'url': None}",Seafood,None,2,NaN,0.0,restaurante-salmones-morelia-suc-siervo,1,1,resy://resy.com/VenueDetails?venue_id=%7B%27re...,{'description': ' Restaurante Salmone's Moreli...,"https://resy.com/?venue_id={'resy': 75788, 'fo..."


In [72]:
df_flat = pd.json_normalize(df['location'])

In [73]:
df = pd.concat([df, df_flat.add_prefix('loc_')], axis=1)

In [74]:
df_nyc = df[df["loc_url_slug"] == 'new-york-ny'].reset_index(drop=True)

In [75]:
df_nyc_flat_url = pd.json_normalize(df_nyc['contact'])
df_nyc_flat_id = pd.json_normalize(df_nyc['id'])

In [76]:
df_nyc = df_nyc.drop(columns=['contact']).join(df_nyc_flat_url)
df_nyc = df_nyc.drop(columns=['id']).join(df_nyc_flat_id)

In [77]:
df_nyc = df_nyc.drop(columns=['location'])

In [78]:
df_nyc.columns

Index(['name', 'type', 'tagline', 'price_range_id', 'rating', 'num_ratings',
       'url_slug', 'enable_discovery', 'enable_for_amex', 'deep_link',
       'metadata', 'web_link', 'loc_address_1', 'loc_address_2',
       'loc_locality', 'loc_region', 'loc_postal_code', 'loc_cross_street_1',
       'loc_cross_street_2', 'loc_longitude', 'loc_latitude',
       'loc_neighborhood', 'loc_time_zone', 'loc_url_slug', 'loc_id',
       'phone_number', 'url', 'resy', 'foursquare', 'google'],
      dtype='object')

In [79]:
enrichdf = df_nyc[["resy", "foursquare", "google", "name", "type", "price_range_id", "rating", "num_ratings", "web_link", "loc_id", "loc_neighborhood", "phone_number", "url"]]

In [80]:
enrichdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2467 entries, 0 to 2466
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   resy              2467 non-null   int64  
 1   foursquare        1391 non-null   object 
 2   google            2466 non-null   object 
 3   name              2467 non-null   object 
 4   type              2467 non-null   object 
 5   price_range_id    2467 non-null   int64  
 6   rating            2439 non-null   float64
 7   num_ratings       2439 non-null   float64
 8   web_link          2467 non-null   object 
 9   loc_id            2467 non-null   object 
 10  loc_neighborhood  2467 non-null   object 
 11  phone_number      2293 non-null   object 
 12  url               2420 non-null   object 
dtypes: float64(2), int64(2), object(9)
memory usage: 250.7+ KB


### Crawl4All 

In [2]:
load_dotenv()

True

In [3]:
nest_asyncio.apply()

In [4]:
client = genai.Client(api_key=os.getenv("GENAI_API_KEY")) 

In [5]:
AI_LIMITER = AsyncLimiter(max_rate=500, time_period=60)

#### Define Data Models

In [6]:
class ClassifiedLink(BaseModel):
    url: str = Field(description="The href URL from the input.")
    text: str = Field(description="The anchor text from the input. May be gibberish.")
    category: Literal['ordering', 'gift_card', 'instagram', 'private_events', 'mailing_list', 'other'] = Field(
        description="The category: 'ordering', 'gift_card', 'instagram', 'private_events', 'mailing_list', or 'other'."
    )

class LinkCollection(BaseModel):
    classified_links: List[ClassifiedLink]

class ClassificationBatch(BaseModel):
    categories: List[Literal['ordering', 'gift_card', 'instagram', 'private_events', 'mailing_list', 'other']]

class RestaurantTechProfile(BaseModel):
    pos_system: Optional[str] = Field(None, description="Inferred POS (Toast, Square, etc)")
    tech_stack: List[str] = Field(default_factory=list, description="Other systems (Bentobox, OpenTable, etc)")
    ordering_provider: Optional[str] = Field(None, description="Who powers the online ordering?")
    instagram_handle: Optional[str] = Field(None, description="The extracted handle (e.g. 'thesmithnyc')")
    newsletter_status: str = Field("Not Found", description="Confirmed or Not Found")
    tripleseat_status: str = Field("Not Found", description="Confirmed, Suspected, or Not Found")


#### Categorization LLM Helper (GeminiFlash)

In [7]:
def classify_links_flash(links: List[dict]) -> LinkCollection:
    """
    Uses gemini-flash-latest (Gemini 2.5) to categorize links 
    instead of guessing with keywords.
    """
    # Prepare batch for LLM
    candidates = links[:100]
    
    #print("candidates links count:", len(candidates))
    
    if not candidates:
        return LinkCollection(classified_links=[])

    # We ask for a Map of Index -> Category to keep response small
    prompt = f"""
    You are a restaurant bot link classifier.
    Classify every one of the following links based on their text and href into:
    - 'ordering' (online ordering, takeout, delivery, 'order now', point of sale, POS)
    - 'gift_card' (Gift cards, merch, store)
    - 'instagram' (Social media links to Instagram)
    - 'private_events' (Private dining, event booking, party reservations)
    - 'mailing_list' (Newsletter signups, email signups, mailing lists)
    - 'other' (Menus, about, contact, locations, reservations)

    If the text is gibberish or empty, RELY MORE on the href URL to classify.

    Input Links:
    {json.dumps(candidates)}
    """

    #print("Flash Classification Prompt:", prompt)

    try:
        response = client.models.generate_content(
            model="gemini-flash-latest", # CURRENTLY GEMINI 2.5 FLASH
            contents=prompt,
            config={
                "temperature": 0.0,
                "response_mime_type": "application/json",
                "response_json_schema": ClassificationBatch.model_json_schema(),
            }
        )

        #print("Flash Classification Response:", response.text)
        
        # Parse map and rebuild list
        batch_result = ClassificationBatch.model_validate_json(response.text)
        final_links = []
        for i, link in enumerate(candidates):
            category = "other" # Default fallback
            if i < len(batch_result.categories):
                category = batch_result.categories[i]
                
            final_links.append(ClassifiedLink(
                url=link.get("href", "") or link.get("url", ""),
                text=link.get("text", ""),
                category=category
            ))

        return LinkCollection(classified_links=final_links)

    except Exception as e:
        print(f"Flash Classification Error: {e}")
        return LinkCollection(classified_links=[])

In [8]:
async def classify_links_flash_async(links: List[dict]) -> LinkCollection:
    """
    Uses gemini-flash-latest (Gemini 2.5) to categorize links async
    """
    candidates = links[:100]
    
    if not candidates:
        return LinkCollection(classified_links=[])

    prompt = f"""
    You are a restaurant bot link classifier.
    Classify every one of the following links based on their text and href into:
    - 'ordering' (online ordering, takeout, delivery, 'order now', point of sale, POS)
    - 'gift_card' (Gift cards, merch, store)
    - 'instagram' (Social media links to Instagram)
    - 'private_events' (Private dining, event booking, party reservations)
    - 'mailing_list' (Newsletter signups, email signups, mailing lists)
    - 'other' (Menus, about, contact, locations, reservations)

    If the text is gibberish or empty, RELY MORE on the href URL to classify.

    Input Links:
    {json.dumps(candidates)}
    """

    async with AI_LIMITER: # Limit rate
        try:
            response = await client.aio.models.generate_content(
                model="gemini-flash-latest", # CURRENTLY GEMINI 2.5 FLASH
                contents=prompt,
                config={
                    "temperature": 0.0,
                    "response_mime_type": "application/json",
                    "response_json_schema": ClassificationBatch.model_json_schema(),
                }
            )

            #print("Flash Classification Response:", response.text)
            
            # Parse map and rebuild list
            batch_result = ClassificationBatch.model_validate_json(response.text)
            final_links = []
            for i, link in enumerate(candidates):
                category = "other"
                if i < len(batch_result.categories):
                    category = batch_result.categories[i]
                    
                final_links.append(ClassifiedLink(
                    url=link.get("href", "") or link.get("url", ""),
                    text=link.get("text", ""),
                    category=category
                ))

            return LinkCollection(classified_links=final_links)

        except Exception as e:
            print(f"Flash Classification Error: {e}")
            return LinkCollection(classified_links=[])

#### Analyze Tech Stack (Gemini3)

In [9]:
def analyze_tech_stack_gemini3(
    classified_links: List[ClassifiedLink], 
    script_domains: List[str], 
    footer_text: str,
    deep_dive_signals: List[str]
) -> RestaurantTechProfile:
    """
    Uses gemini-3-pro-preview to reason about the signals found.
    """
    
    # Organize data for the model
    ordering_urls = [l.url for l in classified_links if l.category == "ordering"]
    gift_urls = [l.url for l in classified_links if l.category == "gift_card"]
    socials = [l.url for l in classified_links if l.category == "instagram"]
    newsletter_links = [l.url for l in classified_links if l.category == "mailing_list"]
    
    prompt = f"""
    Analyze these signals to determine the Restaurant's Tech Stack.
    
    1. Validated Ordering Links: {ordering_urls}
    2. Validated Gift Card Links: {gift_urls}

    3. **Deep Dive Signals (Ordering/Gift Pages):** 
    {json.dumps(deep_dive_signals, indent=2)}
    (IMPORTANT: These are links/redirects found AFTER clicking the ordering/gift buttons. 
     Look here for 3rd party POS domains like 'toasttab.com', 'spoton.com', 'clover.com'.)

    4. Loaded Scripts/Domains: {script_domains}
    5. Footer Text: {footer_text}
    6. Social Links: {socials}
    7. Newsletter/Mailing List Links: {newsletter_links}
    
    Task:
    - Identify the POS System (Point of Sale). 
      - PRIORITY: Look at "Deep Dive Signals". If a link redirects to or points to a known POS (Toast, Square, SpotOn, Upserve, Aloha, Heartland, Clover etc), that is the POS.
      - SECONDARY: Look at Scripts.
    - Identify the technologies the website uses (e.g., Website builders like Bentobox or Squarespace, online order builders like Sauce or Owner.com, reservation systems).
    - Extract the Instagram Handle.
    - Determine Newsletter/Mailing List status (Confirmed, Not Found).
    """

    #print("gemini3", prompt)

    try:
        response = client.models.generate_content(
            model="gemini-flash-latest", # REASONING MODEL
            contents=prompt,
            config={
                "thinking_config": types.ThinkingConfig(thinking_budget=-1),
                "response_mime_type": "application/json",
                "response_json_schema": RestaurantTechProfile.model_json_schema(),
            }
        )

        techprofile = RestaurantTechProfile.model_validate_json(response.text)
        #print("gemini3profile", techprofile)
        return techprofile
    except Exception as e:
        print(f"Gemini 3 Error: {e}")
        return RestaurantTechProfile()

In [10]:
async def analyze_tech_stack_async(
    classified_links: List[ClassifiedLink], 
    script_domains: List[str], 
    footer_text: str,
    deep_dive_signals: List[str]
) -> RestaurantTechProfile:
    """
    Uses thinking model to reason about the signals found.
    """
    
    # Organize data for the model
    ordering_urls = [l.url for l in classified_links if l.category == "ordering"]
    gift_urls = [l.url for l in classified_links if l.category == "gift_card"]
    socials = [l.url for l in classified_links if l.category == "instagram"]
    newsletter_links = [l.url for l in classified_links if l.category == "mailing_list"]
    
    prompt = f"""
    Analyze these signals to determine the Restaurant's Tech Stack.
    
    1. Validated Ordering Links: {ordering_urls}
    2. Validated Gift Card Links: {gift_urls}

    3. **Deep Dive Signals (Ordering/Gift Pages):** 
    {json.dumps(deep_dive_signals, indent=2)}
    (IMPORTANT: These are links/redirects found AFTER clicking the ordering/gift buttons. 
     Look here for 3rd party POS domains like 'toasttab.com', 'spoton.com', 'clover.com'.)

    4. Loaded Scripts/Domains: {script_domains}
    5. Footer Text: {footer_text}
    6. Social Links: {socials}
    7. Newsletter/Mailing List Links: {newsletter_links}
    
    Task:
    - Identify the POS System (Point of Sale). 
      - PRIORITY: Look at "Deep Dive Signals". If a link redirects to or points to a known POS (Toast, Square, SpotOn, Upserve, Aloha, Heartland, Clover etc), that is the POS.
      - SECONDARY: Look at Scripts.
    - Identify the technologies the website uses (e.g., Website builders like Bentobox or Squarespace, online order builders like Sauce or Owner.com, reservation systems).
    - Extract the Instagram Handle.
    - Determine Newsletter/Mailing List status (Confirmed, Not Found).
    """

    async with AI_LIMITER: # Limit concurrency

        try:
            response = await client.aio.models.generate_content(
                model="gemini-flash-latest", # REASONING MODEL
                contents=prompt,
                config={
                    "thinking_config": types.ThinkingConfig(thinking_budget=-1),
                    "response_mime_type": "application/json",
                    "response_json_schema": RestaurantTechProfile.model_json_schema(),
                }
            )

            techprofile = RestaurantTechProfile.model_validate_json(response.text)
            return techprofile
        except Exception as e:
            print(f"Gemini 3 Error: {e}")
            return RestaurantTechProfile()

#### Other Helper Functions

In [11]:
def extract_footer(soup):
    # --- LEVEL 1: The "Smoking Gun" Selectors (Specific Builders) ---
    # These attributes are definitive proof that an element is a footer template.
    high_priority_selectors = [
        '[data-elementor-post-type="footer"]',  # <--- THIS FIXES YOUR SPECIFIC HTML
        '[data-elementor-type="footer"]',       # Standard Elementor
        '.elementor-location-footer',           # Elementor Theme Builder
        '#footer-builder',
        '.fusion-footer',                       # Avada
        '.divi-builder #main-footer',           # Divi
    ]

    for selector in high_priority_selectors:
        element = soup.select_one(selector)
        if element:
            #print(f"Match found via High Priority: {selector}")
            return clean_text(element)

    # --- LEVEL 2: The "Name Game" (Class/ID Search) ---
    # We look for classes/IDs containing "footer", but we filter them.
    # We don't want to accidentally grab a wrapper like <div id="page-wrapper-footer-included">
    candidates = []
    
    # Find ALL divs, sections, and footers
    tags = soup.find_all(['div', 'section', 'footer'])
    
    for tag in tags:
        # Get string of classes and ID
        classes = " ".join(tag.get('class', []))
        id_val = tag.get('id', "")
        identifier = (classes + " " + id_val).lower()
        
        # Check if it identifies as a footer
        if 'footer' in identifier or 'colophon' in identifier:
            # FILTER: Ignore elements that are too large (likely page wrappers)
            # A footer usually has fewer than 2000 characters of text.
            text_len = len(tag.get_text(strip=True))
            if 10 < text_len < 2500:
                candidates.append(tag)

    # If we found candidates, return the LAST one in the DOM (closest to bottom)
    if candidates:
        #print("Match found via Class/ID Search")
        return clean_text(candidates[-1])

    # --- LEVEL 3: Text Anchors (Copyright & Developer Credits) ---
    # Your example didn't have "Copyright", but it had "Site Made With Love".
    # We look for these phrases and grab their parent container.
    keywords = [
        r'©', 
        r'&copy;', 
        r'copyright', 
        r'all rights reserved', 
        r'powered by', 
        r'made with love', 
        r'made by', 
        r'designed by'
    ]
    
    pattern = re.compile('|'.join(keywords), re.IGNORECASE)
    
    # Find text nodes matching keywords
    matches = soup.find_all(string=pattern)
    
    if matches:
        # Get the last match (closest to bottom)
        target = matches[-1]
        
        # Walk up 3 levels to find a container (div or section)
        parent = target.parent
        for _ in range(3):
            if parent.name in ['div', 'section', 'footer', 'aside']:
                #print(f"Match found via Text Anchor: {target.strip()[:20]}...")
                return clean_text(parent)
            parent = parent.parent

    return ""

def clean_text(element):
    """Clean up whitespace but preserve readability"""
    # Get text with newline separators
    text = element.get_text("\n", strip=True)
    # Remove excessive newlines
    text = re.sub(r'\n\s*\n', '\n', text)
    return text

In [12]:
def filter_urls(urls):
     """
    Remove social media URLs and Resy URLs from a list.
    Keep everything else.
    """
     BLOCKED_DOMAINS = [
            "instagram.com",
            "facebook.com",
            "fb.com",
            "tiktok.com",
            "twitter.com",
            "x.com",
            "youtube.com",
            "youtu.be",
            "linkedin.com",
            "pinterest.com",
            "snapchat.com",
            "threads.net",
            "resy.com"
        ]
     
     filtered = []
     for url in urls:
        if not isinstance(url, str):
            continue
        
        url = url.strip()
        if not url:
            continue
        
        netloc = urlparse(url).netloc.lower()
        if any(domain in netloc for domain in BLOCKED_DOMAINS):
            continue
        
        filtered.append(url)

     return filtered

In [13]:
def is_http_url(url: str) -> bool:
    """Checks if URL starts with http:// or https://"""
    if not url: return False
    return url.lower().startswith(('http://', 'https://'))

#### Main Logic

In [29]:
async def process_restaurant(crawler: AsyncWebCrawler, start_url: str, config: CrawlerRunConfig):
    print(f"--- Processing: {start_url} ---")
    
    # 1. Crawl Homepage
    result = await crawler.arun(url=start_url, config=config)
    if not result.success:
        print(f"  ! Failed to crawl: {start_url}")
        return None

    # 2. Extract Basic Signals (Scripts & Footer)
    soup = BeautifulSoup(result.html, 'html.parser')
    scripts = set()
    for s in soup.find_all('script', src=True):
        domain = urlparse(s.get('src')).netloc
        if domain: scripts.add(domain)
    
    footer = extract_footer(soup)
    
    # Check Tripleseat on Homepage
    ts_found = False
    if "tripleseat.com" in result.html or soup.find(id="tripleseat-form"):
        ts_found = True

    # Check Newsletter on Homepage
    newsletter_found = False
    newsletter_keywords = ["newsletter", "mailing list", "subscribe", "sign up", "stay updated"]
    if any(kw in result.html.lower() for kw in newsletter_keywords):
        newsletter_found = True

    # 3. USE GEMINI FLASH: Classify Links
    # We merge internal and external links for classification
    all_links = [{"text": l['text'], "href": urljoin(start_url, l['href'])} 
                 for l in result.links.get('internal', []) + result.links.get('external', [])]
    
    link_collection = classify_links_flash(all_links)
    classified_links = link_collection.classified_links

    # We visit these pages to find:
    # A) Redirects (e.g. /order -> toasttab.com)
    # B) Links ON that page (e.g. /order -> Button href="toasttab.com")
    deep_dive_signals = []
    ordering_candidates = [l for l in classified_links if l.category == "ordering"]
    gift_candidates = [l for l in classified_links if l.category == "gift_card"]
    urls_to_drill = (ordering_candidates + gift_candidates)[:4]

    for link_obj in urls_to_drill:
        print(f"  > Drilling down into tech link: {link_obj.url}")
        try:
            sub_res = await crawler.arun(url=link_obj.url)

            if sub_res.success:
                # Signal A: Did we get redirected?
                # Compare the final URL to the one we clicked.
                # If we clicked /order and ended up on toasttab.com, that's a strong signal.
                if urlparse(sub_res.url).netloc != urlparse(link_obj.url).netloc:
                     deep_dive_signals.append(f"Redirect from {link_obj.text}: {sub_res.url}")

                # Signal B: Scan for External Links on this sub-page
                # This handles the case where the page is internal but contains a button to the POS.
                # We extract external links found on this sub-page.
                external_links = sub_res.links.get("external", [])

                for ext_link in external_links[:10]:
                    href = ext_link.get('href', '')
                    if href:
                        deep_dive_signals.append(f"Link on '{link_obj.text}' page: {href}")

                # Signal C: Capture scripts on this sub-page
                sub_soup = BeautifulSoup(sub_res.html, 'html.parser')
                for s in sub_soup.find_all('script', src=True):
                    domain = urlparse(s.get('src')).netloc
                    if domain: scripts.add(domain)
        except Exception as e:
            print(f"  ! Failed to drill down {link_obj.url}: {e}")
    
    # 4. USE GEMINI 3: Analyze Tech Stack
    # We pass the *clean, categorized* data to the smart model
    tech_profile = analyze_tech_stack_gemini3(
        classified_links, 
        list(scripts)[:50], 
        footer,
        deep_dive_signals
    )

    if ts_found: 
        tech_profile.tripleseat_status = "Confirmed (Homepage)"
    
    if newsletter_found:
        tech_profile.newsletter_status = "Confirmed (Homepage)"

    # 5. Navigate to Private Events (if not already found)
    # We look for the link categorized as 'private_events' by Flash
    events_link = next((l for l in classified_links if l.category == "private_events"), None)
    if events_link:
        print(f"  > Flash identified Events page: {events_link.url}")
        try:
            evt_res = await crawler.arun(url=events_link.url)
            if evt_res.success:
                if "tripleseat.com" in evt_res.html or "tripleseat" in evt_res.html.lower():
                    tech_profile.tripleseat_status = "Confirmed (Events Page)"
                elif tech_profile.tripleseat_status == "Not Found":
                    tech_profile.tripleseat_status = "Not Found on Events Page"
        except Exception as e:
            print(f"  ! Failed to process Events page {events_link.url}: {e}")

    return {
        "url": start_url,
        "pos": tech_profile.pos_system,
        "stack": tech_profile.tech_stack,
        "instagram": tech_profile.instagram_handle,
        "tripleseat": tech_profile.tripleseat_status,
        "newsletter": tech_profile.newsletter_status,
        "ordering_url": next((l.url for l in classified_links if l.category == "ordering"), None)
    }
    

In [152]:
async def main():
    
    urls = ["http://www.nycprimerib.com/"]
    
    results = []
    crawler_run_config = CrawlerRunConfig(cache_mode=CacheMode.BYPASS)
    async with AsyncWebCrawler(verbose=False) as crawler:
        for url in urls:
            try:
                data = await process_restaurant(crawler, url, config=crawler_run_config)
                if data: results.append(data)
            except Exception as e:
                print(f"Error on {url}: {e}")

    df = pd.DataFrame(results)

    final_df = pd.merge(df, enrichdf, on='url', how='left')

    display(final_df)

In [ ]:
await main()

[INIT].... → Crawl4AI 0.7.7 

--- Processing: http://www.nycprimerib.com/ ---


[FETCH]... ↓ http://www.nycprimerib.com/                                                                          |
✓ | ⏱: 1.46s 

[SCRAPE].. ◆ http://www.nycprimerib.com/                                                                          |
✓ | ⏱: 0.01s 

[COMPLETE] ● http://www.nycprimerib.com/                                                                          |
✓ | ⏱: 1.47s 

...

  > Flash identified Events page: https://www.nycprimerib.com/private-dining


[FETCH]... ↓ https://www.nycprimerib.com/private-dining                                                           |
✓ | ⏱: 1.37s 

[SCRAPE].. ◆ https://www.nycprimerib.com/private-dining                                                           |
✓ | ⏱: 0.02s 

[COMPLETE] ● https://www.nycprimerib.com/private-dining                                                           |
✓ | ⏱: 1.39s 

,url,pos,stack,instagram,tripleseat,newsletter,ordering_url,resy,foursquare,google,name,type,price_range_id,rating,num_ratings,web_link,loc_id,loc_neighborhood,phone_number
0,http://www.nycprimerib.com/,None,"[Squarespace, Resy, Harri, Secure Tree]",nycprimerib,Confirmed (Events Page),Confirmed (Homepage),https://www.nycprimerib.com/cart,834,585164b77220e62219c9aeb6,ChIJ-7AOKpRZwokRMq0XnG_eehU,4 Charles Prime Rib,Steakhouse,3,4.94094,28952.0,"https://resy.com/?venue_id={'resy': 834, 'four...",ny,West Village,None


#### Concurrent

##### Prepare Data for Concurrent Processing

In [43]:
urlset = set(enrichdf["url"].to_list())

In [45]:
urllist = filter_urls(list(urlset))

In [46]:
allurls = sorted(urllist)

In [65]:
first100 = allurls[:100]
second400 = allurls[100:500]

##### SYNCHRONOUS HELPERS

In [14]:
def _sync_extract_homepage_signals(html: str):
    """
    CPU-bound task: Parses homepage HTML to find scripts, footer, and Tripleseat.
    """
    soup = BeautifulSoup(html, 'lxml')
    
    # Extract Scripts
    scripts = set()
    for s in soup.find_all('script', src=True):
        domain = urlparse(s.get('src')).netloc
        if domain: scripts.add(domain)
    
    # Extract Footer
    footer = extract_footer(soup)
    
    # Check Tripleseat
    # Note: simple string check is fast, but finding ID requires soup
    ts_found = "tripleseat.com" in html or soup.find(id="tripleseat-form") is not None

    newsletter_found = False
    newsletter_keywords = ["newsletter", "mailing list", "subscribe", "sign up", "stay updated"]
    if any(kw in html.lower() for kw in newsletter_keywords):
        newsletter_found = True
    
    return scripts, footer, ts_found, newsletter_found

def _sync_extract_scripts_only(html: str):
    """
    CPU-bound task: Parses sub-page HTML just for scripts.
    """
    soup = BeautifulSoup(html, 'lxml')
    scripts = set()
    for s in soup.find_all('script', src=True):
        domain = urlparse(s.get('src')).netloc
        if domain: scripts.add(domain)
    return scripts

##### MAIN CONCURRENT LOGIC

In [15]:
async def process_restaurant_concurrent(crawler: AsyncWebCrawler, start_url: str, config: CrawlerRunConfig):
    print(f"--- Processing: {start_url} ---")

    if not is_http_url(start_url):
        print(f"  ! Invalid URL scheme: {start_url}")
        return None
    
    # 1. Crawl Homepage
    try:
        result = await crawler.arun(url=start_url, config=config)
    except Exception as e:
        # Catch DNS errors, Timeouts, etc.
        print(f"  ! Critical crawl failure on {start_url}: {str(e)[:100]}...")
        return None
    
    if not result.success:
        print(f"  ! Failed to crawl: {start_url} (Status: {result.status_code})")
        return None

    # 2. Extract Basic Signals (CPU Bound - Offload to Thread)
    # This prevents blocking the event loop while parsing 
    scripts, footer, ts_found, newsletter_found = await asyncio.to_thread(
        _sync_extract_homepage_signals, 
        result.html
    )

    # 3. USE GEMINI FLASH: Classify Links
    # We merge internal and external links for classification
    all_links = []
    raw_links = result.links.get('internal', []) + result.links.get('external', [])
    for l in raw_links:
        full_url = urljoin(start_url, l['href'])
        if is_http_url(full_url):
            all_links.append({"text": l['text'], "href": full_url})
    
    link_collection = await classify_links_flash_async(all_links)
    classified_links = link_collection.classified_links

    # We visit these pages to find:
    # A) Redirects (e.g. /order -> toasttab.com)
    # B) Links ON that page (e.g. /order -> Button href="toasttab.com")
    ordering_candidates = [l for l in classified_links if l.category == "ordering" and is_http_url(l.url)]
    gift_candidates = [l for l in classified_links if l.category == "gift_card" and is_http_url(l.url)]
    urls_to_drill_objs = (ordering_candidates + gift_candidates)[:4]
    urls_to_drill_strings = [u.url for u in urls_to_drill_objs]

    deep_dive_signals = []
    
    if urls_to_drill_strings:
        print(f"  > Drilling down {len(urls_to_drill_strings)} URLs for {start_url}...")
        
        # Use MemoryAdaptiveDispatcher to handle these sub-tasks safely in parallel
        # This prevents the browser from crashing if we are running 5 restaurants * 5 sub-links = 25 tabs
        sub_dispatcher = MemoryAdaptiveDispatcher(
            memory_threshold_percent=90.0,
            max_session_permit=5,
            rate_limiter=CrawlRateLimiter(base_delay=(0.5, 1.0))
        )
        try:
            sub_results = await crawler.arun_many(
                urls=urls_to_drill_strings,
                config=config,
                dispatcher=sub_dispatcher
            )

            for i, sub_res in enumerate(sub_results):
                original_link_obj = urls_to_drill_objs[i]
                if sub_res.success:
                    # Redirect check
                    if urlparse(sub_res.url).netloc != urlparse(original_link_obj.url).netloc:
                        deep_dive_signals.append(f"Redirect from {original_link_obj.text}: {sub_res.url}")
                    # Link check
                    external_links = sub_res.links.get("external", [])
                    for ext_link in external_links[:10]:
                        href = ext_link.get('href', '')
                        if href: deep_dive_signals.append(f"Link on '{original_link_obj.text}' page: {href}")
                    # Script check
                    new_scripts = await asyncio.to_thread(_sync_extract_scripts_only, sub_res.html)
                    scripts.update(new_scripts)
                else:
                    if sub_res.url and sub_res.url != original_link_obj.url and is_http_url(sub_res.url):
                        # Use the final URL as a signal even if we have no HTML
                        deep_dive_signals.append(f"Redirect (Trace) from {original_link_obj.text}: {sub_res.url}")

        except Exception as e:
            print(f"  ! Failed during sub-page drills for {start_url}: {e}")

    
    # 4. Analyze Tech Stack
    tech_profile = await analyze_tech_stack_async(
        classified_links, 
        list(scripts)[:50], 
        footer,
        deep_dive_signals
    )

    if ts_found: tech_profile.tripleseat_status = "Confirmed (Homepage)"
    
    if newsletter_found: tech_profile.newsletter_status = "Confirmed (Homepage)"

    # 5. Navigate to Private Events (if not already found)
    # We look for the link categorized as 'private_events' by Flash
    events_link = next((l for l in classified_links if l.category == "private_events" and is_http_url(l.url)), None)
    if events_link:
        print(f"  > Flash identified Events page: {events_link.url}")
        try:
            evt_res = await crawler.arun(url=events_link.url)
            if evt_res.success:
                if "tripleseat.com" in evt_res.html or "tripleseat" in evt_res.html.lower():
                    tech_profile.tripleseat_status = "Confirmed (Events Page)"
                elif tech_profile.tripleseat_status == "Not Found":
                    tech_profile.tripleseat_status = "Not Found on Events Page"
        except Exception as e:
            print(f"  ! Failed to process Events page {events_link.url}: {e}")

    return {
        "url": start_url,
        "pos": tech_profile.pos_system,
        "stack": tech_profile.tech_stack,
        "instagram": tech_profile.instagram_handle,
        "tripleseat": tech_profile.tripleseat_status,
        "newsletter": tech_profile.newsletter_status,
        "ordering_url": next((l.url for l in classified_links if l.category == "ordering"), None)
    }
    

In [ ]:
async def main_concurrent():

    urls = ["https://www.rosesluxury.com/", "https://www.crownshy.nyc/", "https://estelanyc.com/", "https://cosmenyc.com/", "https://www.le-bernardin.com/", "https://www.elevenmadisonpark.com/"]

    output_file = "restaurant_results.csv"

    if not os.path.exists(output_file):
        pd.DataFrame(columns=["url", "pos", "stack", "instagram", "tripleseat", "newsletter", "ordering_url"]).to_csv(output_file, index=False)
        print(f"Created new file: {output_file}")
    else:
        print(f"Appending to existing file: {output_file}")
    
    browser_config = BrowserConfig(headless=True, verbose=False, text_mode=True)
    crawler_run_config = CrawlerRunConfig(cache_mode=CacheMode.BYPASS, page_timeout=20000)

    concurrency_sem = asyncio.Semaphore(5)

    async def semaphore_worker(url, crawler):
        async with concurrency_sem:
            try:
                return await process_restaurant_concurrent(crawler, url, crawler_run_config)
            except Exception as e:
                print(f"Error processing {url}: {e}")
                return None
    
    async with AsyncWebCrawler(config=browser_config) as crawler:
        # Create tasks
        tasks = [semaphore_worker(url, crawler) for url in urls]
        
        print(f"Processing {len(urls)} restaurants with incremental saving...")
        
        # as_completed yields futures as they finish
        for future in asyncio.as_completed(tasks):
            result = await future
            
            if result:
                print(f"Finished: {result['url']} -> Saving.")
                
                # INCREMENTAL SAVE: Create 1-row DF and append to CSV
                df_chunk = pd.DataFrame([result])
                
                # Check if file exists to determine if we need a header (in case it was deleted mid-run)
                use_header = not os.path.exists(output_file)
                df_chunk.to_csv(output_file, mode='a', header=use_header, index=False)
                
            else:
                print(f"A task: {future} failed returned None.")

    print("\n--- Processing Complete ---")
    if os.path.exists(output_file):
        final_df = pd.read_csv(output_file)
        display(final_df.tail(6))
        #print(final_df.to_string())

In [17]:
await main_concurrent()

Appending to existing file: restaurant_results.csv
Processing 6 restaurants with incremental saving...
--- Processing: https://www.rosesluxury.com/ ---
--- Processing: https://estelanyc.com/ ---
--- Processing: https://www.le-bernardin.com/ ---
--- Processing: https://www.crownshy.nyc/ ---
--- Processing: https://cosmenyc.com/ ---


[FETCH]... ↓ https://estelanyc.com/                                                                               |
✓ | ⏱: 1.50s 

[SCRAPE].. ◆ https://estelanyc.com/                                                                               |
✓ | ⏱: 0.00s 

[COMPLETE] ● https://estelanyc.com/                                                                               |
✓ | ⏱: 1.51s 

[FETCH]... ↓ https://www.rosesluxury.com/                                                                         |
✓ | ⏱: 1.64s 

...

  > Flash identified Events page: https://www.elevenmadisonpark.com/events


[FETCH]... ↓ https://www.elevenmadisonpark.com/events                                                             |
✓ | ⏱: 0.60s 

[SCRAPE].. ◆ https://www.elevenmadisonpark.com/events                                                             |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://www.elevenmadisonpark.com/events                                                             |
✓ | ⏱: 0.61s 

Finished: https://www.elevenmadisonpark.com/ -> Saving.

--- Processing Complete ---
                                                                                                                                                                                                    url            pos                                                                                                  stack                 instagram                tripleseat            newsletter                                                                                                                                                                                                                                                                                                                                                                                                                  ordering_url
0                                                                                                            

#### Tester

In [24]:
urls = [
        "https://www.andsonnyc.com",
        "https://www.rezdora.nyc/"
    ]
    
results = []
crawler_run_config = CrawlerRunConfig(cache_mode=CacheMode.BYPASS)
async with AsyncWebCrawler(verbose=False) as crawler:
    for url in urls:
        try:
            data = await process_restaurant(crawler, url, config=crawler_run_config)
            if data: results.append(data)
        except Exception as e:
            print(f"Error on {url}: {e}")

df = pd.DataFrame(results)
display(df)

[INIT].... → Crawl4AI 0.7.7 

--- Processing: https://www.andsonnyc.com ---


[FETCH]... ↓ https://www.andsonnyc.com                                                                            |
✓ | ⏱: 1.15s 

[SCRAPE].. ◆ https://www.andsonnyc.com                                                                            |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://www.andsonnyc.com                                                                            |
✓ | ⏱: 1.16s 

--- Processing: https://www.rezdora.nyc/ ---


[FETCH]... ↓ https://www.rezdora.nyc/                                                                             |
✓ | ⏱: 1.24s 

[SCRAPE].. ◆ https://www.rezdora.nyc/                                                                             |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://www.rezdora.nyc/                                                                             |
✓ | ⏱: 1.25s 

,result,soup,scripts,footer,ts_found
0,(url='https://www.andsonnyc.com' html='<!DOCTY...,"[html, [[<meta content=""A7vZI3v+Gz7JfuRolKNM4A...","{www.googletagmanager.com, cdn.jsdelivr.net, u...",Greenwich Village 62 West 9th Street between 5...,False
1,(url='https://www.rezdora.nyc/' html='<!DOCTYP...,"[html, [[<meta content=""A7vZI3v+Gz7JfuRolKNM4A...","{app-assets.getbento.com, theme-assets.getbent...",Facebook Twitter Instagram Hours & Location Me...,False


In [25]:
all_links = [{"text": l['text'], "href": urljoin("https://www.rezdora.nyc/", l['href'])} 
                 for l in df["result"][1].links.get('internal', []) + df["result"][1].links.get('external', [])]

In [26]:
classify_test_results = classify_links_flash(all_links)

In [27]:
test_links = classify_test_results.classified_links

In [33]:
deep_dive_signals = []
scripts = set()
ordering_candidates = [l for l in test_links if l.category == "ordering"]
gift_candidates = [l for l in test_links if l.category == "gift_card"]
urls_to_drill = (ordering_candidates + gift_candidates)[:4]

In [37]:
for link_obj in urls_to_drill:
        print(f"  > Drilling down into tech link: {link_obj.url}")
        try:
            async with AsyncWebCrawler(verbose=False) as crawler:
                sub_res = await crawler.arun(url=link_obj.url, config=crawler_run_config)

                if sub_res.success:
                    # Signal A: Did we get redirected?
                    # Compare the final URL to the one we clicked.
                    # If we clicked /order and ended up on toasttab.com, that's a strong signal.
                    if urlparse(sub_res.url).netloc != urlparse(link_obj.url).netloc:
                        deep_dive_signals.append(f"Redirect from {link_obj.text}: {sub_res.url}")

                    # Signal B: Scan for External Links on this sub-page
                    # This handles the case where the page is internal but contains a button to the POS.
                    # We extract external links found on this sub-page.
                    external_links = sub_res.links.get("external", [])

                    for ext_link in external_links[:15]:
                        href = ext_link.get('href', '')
                        if href:
                            deep_dive_signals.append(f"Link on '{link_obj.text}' page: {href}")

                    # Signal C: Capture scripts on this sub-page
                    sub_soup = BeautifulSoup(sub_res.html, 'html.parser')
                    for s in sub_soup.find_all('script', src=True):
                        domain = urlparse(s.get('src')).netloc
                        if domain: scripts.add(domain)
        except Exception as e:
            print(f"  ! Failed to drill down {link_obj.url}: {e}")

  > Drilling down into tech link: https://www.rezdora.nyc/gift-cards


[INIT].... → Crawl4AI 0.7.7 

[FETCH]... ↓ https://www.rezdora.nyc/gift-cards                                                                   |
✓ | ⏱: 1.11s 

[SCRAPE].. ◆ https://www.rezdora.nyc/gift-cards                                                                   |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://www.rezdora.nyc/gift-cards                                                                   |
✓ | ⏱: 1.12s 

In [41]:
tech_profile = analyze_tech_stack_gemini3(
        test_links, 
        list(scripts)[:50], 
        results[1]['footer'],
        deep_dive_signals # <--- Passing the deep dive content
    )

In [43]:
events_link = next((l for l in test_links if l.category == "private_events"), None)
if events_link:
    print(f"  > Flash identified Events page: {events_link.url}")
    try:
        async with AsyncWebCrawler(verbose=False) as crawler:
            evt_res = await crawler.arun(url=events_link.url)
            if evt_res.success:
                if "tripleseat.com" in evt_res.html or "tripleseat" in evt_res.html.lower():
                    tech_profile.tripleseat_status = "Confirmed (Events Page)"
                elif tech_profile.tripleseat_status == "Not Found":
                    tech_profile.tripleseat_status = "Not Found on Events Page"
    except Exception as e:
            print(f"  ! Failed to drill down {events_link.url}: {e}")

  > Flash identified Events page: https://www.rezdora.nyc/private-events


[INIT].... → Crawl4AI 0.7.7 

[FETCH]... ↓ https://www.rezdora.nyc/private-events                                                               |
✓ | ⏱: 1.95s 

[SCRAPE].. ◆ https://www.rezdora.nyc/private-events                                                               |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://www.rezdora.nyc/private-events                                                               |
✓ | ⏱: 1.96s 

### Google SERP Reviews Call

In [65]:
from serpapi.google_search import GoogleSearch
import time

In [34]:
crawleddf = pd.read_csv("restaurant_results.csv")

In [35]:
crawleddf.head()

,url,pos,stack,instagram,tripleseat,newsletter,ordering_url
0,https://rezdora.nyc/,Toast,"['Bentobox', 'Resy']",rezdoranyc,Confirmed (Events Page),Confirmed (Homepage),NaN
1,http://12chairscafe.com/,Toast,"['Bentobox', 'Sauce', 'Resy']",12chairscafe,Not Found on Events Page,Confirmed (Homepage),https://www.12chairscafe.com/catering
2,http://www.nycprimerib.com/,NaN,"['Squarespace', 'Resy', 'SecureTree', 'Harri']",nycprimerib,Confirmed (Events Page),Confirmed (Homepage),https://www.nycprimerib.com/cart
3,http://www.390social.com,NaN,"['Squarespace', 'Resy', 'Perfect Venue']",390social,Not Found on Events Page,Confirmed (Homepage),https://www.390social.com/cart
4,https://www.21greenpoint.com/,NaN,"['BentoBox', 'Resy']",21greenpoint,Not Found on Events Page,Confirmed (Homepage),NaN


In [36]:
place_iddf = enrichdf[["url", "google"]].dropna().drop_duplicates()

In [37]:
serpapi_test = pd.merge(crawleddf, place_iddf, on="url", how="left")

In [41]:
serpapi_first_100 = serpapi_test[serpapi_test['google'].notnull()].head(100)

In [60]:
serp_target_list = serpapi_first_100["google"].to_list()
demo = [serp_target_list[0]]

In [67]:
data_rows = []

In [68]:
print(f"Starting processing for {len(serp_target_list)} places...")
for place_id in serp_target_list:
    time.sleep(1)
    params = {
        "api_key": os.getenv("SERPAPI_API_KEY"),
        "engine": "google_maps_reviews",
        "place_id": place_id,
        "sort_by": "qualityScore",  # Default sort often bubbles up interactions
    }

    try:
        search = GoogleSearch(params)
        raw_results = search.get_dict()

        reviews = raw_results.get("reviews", [])

        has_response = False
        for review in reviews:
            if "response" in review and review["response"]:
                has_response = True
                break
        
        data_rows.append({
            "place_id": place_id,
            "has_owner_response": has_response,
            "raw_review_page": raw_results  # Stores the entire JSON dict
        })

        print(f"Processed {place_id}: {'Responded' if has_response else 'No response found'}")
    
    except Exception as e:
        print(f"Error processing {place_id}: {e}")
        # Log the error in the dataframe as well to keep row count consistent
        data_rows.append({
            "place_id": place_id,
            "has_owner_response": False,
            "raw_review_page": {"error": str(e)}
        })
    
serp_df_final = pd.DataFrame(data_rows)

print("\nDataFrame Created:")
display(serp_df_final.head())

Starting processing for 100 places...
Processed ChIJDdKSyCVZwokR-9svvc9YDFo: No response found
Processed ChIJbZsGko1ZwokR_Q1lznmBuU8: Responded
Processed ChIJ-7AOKpRZwokRMq0XnG_eehU: No response found
Processed ChIJMVaP2v1awokRB2I4NBXaspU: No response found
Processed ChIJp4jIhmpZwokRabZfKVJPoBY: Responded
Processed ChIJR4I0f-5YwokRK4ta-to3u2w: No response found
Processed ChIJ5T2ntSlawokRZpwIhBG3oxM: Responded
Processed ChIJ4Q2rG4JZwokR8TVjhflNfMk: No response found
Processed ChIJoSqkgRZawokR_ZAqr3pOMO8: Responded
Error processing ChIJmSvG_ZFZwokRTOFeiLXzkmA: HTTPSConnectionPool(host='serpapi.com', port=443): Max retries exceeded with url: /search?api_key=2bd993df856234791a1dc7a302e38d0c522d1a1cb1a4f74159190707662c7ae3&engine=google_maps_reviews&place_id=ChIJmSvG_ZFZwokRTOFeiLXzkmA&sort_by=qualityScore&output=json&source=python (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x3224165d0>: Failed to resolve 'serpapi.com' ([Errno 8] nodename nor servname prov

,place_id,has_owner_response,raw_review_page
0,ChIJDdKSyCVZwokR-9svvc9YDFo,False,{'search_metadata': {'id': '69362709550cc48b16...
1,ChIJbZsGko1ZwokR_Q1lznmBuU8,True,{'search_metadata': {'id': '69362939ff3b7a170d...
2,ChIJ-7AOKpRZwokRMq0XnG_eehU,False,{'search_metadata': {'id': '6936293c4c437ff921...
3,ChIJMVaP2v1awokRB2I4NBXaspU,False,{'search_metadata': {'id': '6936293ebf04d5db2e...
4,ChIJp4jIhmpZwokRabZfKVJPoBY,True,{'search_metadata': {'id': '69362940dc74d0be6e...


In [69]:
serp_df_final.to_csv("google_reviews_responses.csv", index=False)

### Merge

In [89]:
reviewsdf = pd.read_csv("google_reviews_responses.csv")
crawldf = pd.read_csv("restaurant_results.csv")
apollodf = pd.read_csv("apollo-accounts-export.csv")

In [90]:
reviewsdf.rename(columns={'place_id': 'google'}, inplace=True)

In [91]:
add_crawl = pd.merge(crawldf, enrichdf, on='url', how='left')

In [92]:
add_review = pd.merge(add_crawl, reviewsdf, on='google', how='left')

In [93]:
from urllib.parse import urlparse

def extract_domain(u):
    if pd.isna(u):
        return u
    return urlparse(u).netloc.lower().lstrip("www.")

In [97]:
apollodf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 488 entries, 0 to 487
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Company Name                488 non-null    object 
 1   Company Name for Emails     488 non-null    object 
 2   Account Stage               488 non-null    object 
 3   Lists                       488 non-null    object 
 4   # Employees                 301 non-null    float64
 5   Industry                    228 non-null    object 
 6   Account Owner               488 non-null    object 
 7   Website                     488 non-null    object 
 8   Company Linkedin Url        278 non-null    object 
 9   Facebook Url                127 non-null    object 
 10  Twitter Url                 69 non-null     object 
 11  Company Street              0 non-null      float64
 12  Company City                0 non-null      float64
 13  Company State               0 non-n

In [95]:
add_review["domain"] = add_review["url"].apply(extract_domain)
apollodf["domain"] = apollodf["Website"].apply(extract_domain)

In [96]:
apollodf["in_apollo"] = True

In [98]:
apollo_subset = apollodf[['domain', 'in_apollo', '# Employees', 'Company Linkedin Url', 'Technologies', 'Annual Revenue']] 

In [99]:
add_apollo = pd.merge(add_review, apollo_subset, on='domain', how='left')

In [101]:
add_apollo.to_csv("final_restaurant_tech_profile.csv", index=False)